<a href="https://colab.research.google.com/github/feist000/Study/blob/master/Text_Mining/TDM_DTM_TF_IDF_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이전 원핫코딩(One-hot encoding) 파일에서는 원핫인코딩이 무엇인지를 정의와 실습을 통해 파악하고, Sklearn과 Gensim을 이용한 BOW를 만들어보았다. 이번에는 BOW(Bag Of Words)의 방법 중 하나인 DTM과 TDM에 대해서 배워보려고 한다. DTM과 TDM은 문서의 등장하는 각 단어의 등장빈도를 행렬로 표현한 것이다. 

# 1) DTM과 TDM

- BOW(Bag of words)의 방법 중 하나
- 문서에 등장하는 각 단어의 등장빈도를 '행렬'로 표현


## TDM 직접 구현 

In [38]:
docs = ['동물원 코끼리',
        '동물원 원숭이 바나나',
        '엄마 코끼리 아기 코끼리',
        '원숭이 바나나 코끼리 바나나']

In [39]:
doc_ls = [] # 리스트에 넣어주기 위한 바구니를 만들자
for doc in docs :
  doc_ls.append(doc.split(' ')) # 공백 단위로 split

doc_ls

[['동물원', '코끼리'],
 ['동물원', '원숭이', '바나나'],
 ['엄마', '코끼리', '아기', '코끼리'],
 ['원숭이', '바나나', '코끼리', '바나나']]

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
docs = ['동물원 코끼리',
        '동물원 원숭이 바나나',
        '엄마 코끼리 아기 코끼리',
        '원숭이 바나나 코끼리 바나나']
        
vector = CountVectorizer()

In [41]:
print(vector.fit_transform(docs).toarray())
# docs로부터 각 단어의 빈도 수를 기록한다.
print(vector.vocabulary_)
# 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[1 0 0 0 0 1]
 [1 1 0 0 1 0]
 [0 0 1 1 0 2]
 [0 2 0 0 1 1]]
{'동물원': 0, '코끼리': 5, '원숭이': 4, '바나나': 1, '엄마': 3, '아기': 2}


방법 2) 기존 방식과 같이 for 문 이용한 TDM

In [42]:
docs = ['동물원 코끼리',
        '동물원 원숭이 바나나',
        '엄마 코끼리 아기 코끼리',
        '원숭이 바나나 코끼리 바나나']

In [43]:
doc_ls = []
for doc in docs :
  doc_ls.append(doc.split(' '))

doc_ls

[['동물원', '코끼리'],
 ['동물원', '원숭이', '바나나'],
 ['엄마', '코끼리', '아기', '코끼리'],
 ['원숭이', '바나나', '코끼리', '바나나']]

In [44]:
from collections import defaultdict
word3id=defaultdict(lambda:len(word3id))
for doc in doc_ls :
  for token in doc:
    word3id[token]

In [45]:
import numpy as np 

TDM =np.zeros((len(word3id), len(doc_ls)),dtype=int) # 기본 틀로서 전부 0으로 된 array를 만든다.
print(TDM)
for i, doc in enumerate(doc_ls):
  for token in doc : 
    TDM[word3id[token],i] += 1 # 해당 토큰의 위치(Column)
# 행렬로 표기(BOW와 차이점 : BOW는 1차원 배열이지만 TDM은 다차원 행렬)
print(TDM)

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[1 1 0 0]
 [1 0 2 1]
 [0 1 0 1]
 [0 1 0 2]
 [0 0 1 0]
 [0 0 1 0]]


In [46]:
import pandas as pd

doc_names = ['문서'+str(i) for i in range(len(doc_ls))]
print('doc_names', doc_names)
sorted_vocab = sorted((value,key) for key, value in word3id.items())
vocab = [v[1] for v in sorted_vocab]

# 여기서부터는 시각화를 위한 단계라 필요에 따라 빼도 괜찮다!
df_TDM = pd.DataFrame(TDM, columns=doc_names)
df_TDM['단어'] = vocab
df_TDM.set_index('단어')

doc_names ['문서0', '문서1', '문서2', '문서3']


,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
원숭이,0,1,0,1
바나나,0,1,0,2
엄마,0,0,1,0
아기,0,0,1,0


방법 3) gensim을 이용한 TDM 

In [47]:
docs = ['동물원 코끼리',
        '동물원 원숭이 바나나',
        '엄마 코끼리 아기 코끼리',
        '원숭이 바나나 코끼리 바나나']

In [48]:
import gensim
from gensim import corpora

doc_ls = []
for doc in docs :
  doc_ls.append(doc.split(' '))

id2word = corpora.Dictionary(doc_ls)
TDM = [id2word.doc2bow(doc) for doc in doc_ls]
TDM

[[(0, 1), (1, 1)],
 [(0, 1), (2, 1), (3, 1)],
 [(1, 2), (4, 1), (5, 1)],
 [(1, 1), (2, 2), (3, 1)]]

In [49]:
from gensim.matutils import sparse2full
import pandas as pd
import numpy as np
doc_names = ['문서'+str(i) for i in range(len(doc_ls))]
vocab = [id2word[i] for i in id2word.keys()]
DTM_matrix = [sparse2full(doc,len(vocab)).tolist() for doc in TDM]


# 시각화
df_TDM = pd.DataFrame(np.array(DTM_matrix, dtype=int).T,
                      columns= doc_names)

df_TDM['단어']= vocab
df_TDM.set_index('단어')

,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
바나나,0,1,0,2
원숭이,0,1,0,1
아기,0,0,1,0
엄마,0,0,1,0


TDM의 한계 


*   단어의 순서를 고려하지 않는다.
*   TDM은 spare하다. 즉, 벡터공간의 낭비, 연산 비효율서을 초래한다.

*   단어 빈도수가 곧 중요도를 의미하지는 않는다.







# 2) TF-IDF

: TDM 내 각 단어의 중요성을 가중치로 표한한 방법. TDM을 사용하는 것보다 더 정확하게 문서비교가 가능. TDM의 단점을 보완한 개정판 느낌. 


> TF는 단어빈도, IDF는 역문서빈도다. 즉, 문서집합에서 단어와 문서의 관련성을 평가하는 방법으로 TF(단어빈도)와 IDF(역문서빈도)를 곱하여 계산한다. 
이를 이용해 단어의 상대적 중요도를 계산할 수 있다. 


> ● TF(Trem Frequency) : 단어빈도

1.   한 문서 내에서 단어가 얼마나 많이 등장했는가
2.   단어가 문서내에 많이 등장할 수록 높은 TF 값을 가진다. 자주 등장할 수록 중요한 단어라는 의미를 가지게 되는 것.




> ● IDF(Inverse Document Frequency) : 역문서빈도


1.   단어가 얼마나 많은 문서에 등장했는지를 나타내는 DF의 역수
2.   단어가 여러문서에 많이 등장 할 수록 IDF는 작아진다. (헷갈리기 쉬움)







> 결론 :  TF값과 IDF값이 둘 다 높은 게 중요한 단어인 것이다. 





## TF-IDF 직접 구현

In [50]:
docs = ['오늘 동물원에서 원숭이와 코끼리를 봤어',
       '동물원에서 원숭이에게 바나나를 줬어 바나나를']

In [51]:
doc_ls = [] # 리스트에 담아주려고 바구니 만들기
for doc in docs :
  doc_ls.append(doc.split()) # 공백 기준으로 split
doc_ls

[['오늘', '동물원에서', '원숭이와', '코끼리를', '봤어'],
 ['동물원에서', '원숭이에게', '바나나를', '줬어', '바나나를']]

In [52]:
from collections import defaultdict
word2id=defaultdict(lambda:len(word2id))
for doc in doc_ls :
  for token in doc:
    word2id[token]

word2id

defaultdict(<function __main__.<lambda>>,
            {'동물원에서': 1,
             '바나나를': 6,
             '봤어': 4,
             '오늘': 0,
             '원숭이에게': 5,
             '원숭이와': 2,
             '줬어': 7,
             '코끼리를': 3})

In [53]:
import numpy as np 

DTM =np.zeros((len(doc_ls),len(word2id)),dtype=int) # 기본 틀로서 전부 0으로 된 array를 만든다.
print(DTM)
for i, doc in enumerate(doc_ls):
  for token in doc : 
    DTM[i, word2id[token]] += 1 # 해당 토큰의 위치(Column)
# 행렬로 표기(BOW와 차이점 : BOW는 1차원 배열이지만 TMD, DTM은 다차원 행렬)
print(DTM)

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[1 1 1 1 1 0 0 0]
 [0 1 0 0 0 1 2 1]]


In [55]:
DTM[0].sum()

5

In [56]:
def computeTF(DTM):
  doc_len = len(DTM) # 문서 갯수 2개
  word_len = len(DTM[0]) # 토큰의 갯수 8개
  # tf 계산하기 전 0으로 셋팅
  tf = np.zeros((doc_len, word_len))
  print(tf)
  # TF 계산 특정단어등장빈도/문서내 전체등장단어빈도
  for doc_i in range(doc_len):
    for word_i in range(word_len):
      tf[doc_i, word_i]=DTM[doc_i,word_i]/DTM[doc_i].sum()
  return tf

In [57]:
computeTF(DTM)

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


array([[0.2, 0.2, 0.2, 0.2, 0.2, 0. , 0. , 0. ],
       [0. , 0.2, 0. , 0. , 0. , 0.2, 0.4, 0.2]])

IDF 계산 시작 

In [58]:
import math 
# IDF 계산 : log(총문서수/단어가등장한문서수)
def computeIDF(DTM):
  doc_len = len(DTM)
  word_len=len(DTM[0])

  idf = np.zeros(word_len)

  for i in range(word_len):
    idf[i] = math.log10(doc_len/np.count_nonzero(DTM[:,i]))
  return idf

In [59]:
computeIDF(DTM)

array([0.30103, 0.     , 0.30103, 0.30103, 0.30103, 0.30103, 0.30103,
       0.30103])

In [66]:
# TF-IDF 곱

def computeTFIDF(DTM):
  tf = computeTF(DTM)
  idf=computeIDF(DTM)
  tfidf = np.zeros(tf.shape)
  for doc_i in range(tf.shape[0]):
    for word_i in range(tf.shape[1]):
      tfidf[doc_i, word_i]=tf[doc_i,word_i]* idf[word_i]
  return tfidf 

In [67]:
computeTFIDF(DTM)

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


array([[0.060206, 0.      , 0.060206, 0.060206, 0.060206, 0.      ,
        0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      , 0.060206,
        0.120412, 0.060206]])

In [68]:
import pandas as pd

sorted_vocab = sorted((value,key) for key, value in word2id.items())
print(sorted_vocab)
vocab = [v[1] for v in sorted_vocab]
print(vocab)
tfidf=computeTFIDF(DTM)

# 시각화
pd.DataFrame(tfidf,columns=vocab)

[(0, '오늘'), (1, '동물원에서'), (2, '원숭이와'), (3, '코끼리를'), (4, '봤어'), (5, '원숭이에게'), (6, '바나나를'), (7, '줬어')]
['오늘', '동물원에서', '원숭이와', '코끼리를', '봤어', '원숭이에게', '바나나를', '줬어']
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


,오늘,동물원에서,원숭이와,코끼리를,봤어,원숭이에게,바나나를,줬어
0,0.060206,0.0,0.060206,0.060206,0.060206,0.000000,0.000000,0.000000
1,0.000000,0.0,0.000000,0.000000,0.000000,0.060206,0.120412,0.060206


## Sklearn 사용

In [70]:
docs = ['오늘 동물원에서 원숭이와 코끼리를 봤어',
       '동물원에서 원숭이에게 바나나를 줬어 바나나를']

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidv = TfidfVectorizer()
tfidf = tfidv.fit(docs)
tfidf.transform(docs).toarray()
vocab = tfidf.get_feature_names()

In [72]:
vocab

['동물원에서', '바나나를', '봤어', '오늘', '원숭이에게', '원숭이와', '줬어', '코끼리를']

In [73]:
tfidv.transform(docs).toarray()[0]

array([0.33517574, 0.        , 0.47107781, 0.47107781, 0.        ,
       0.47107781, 0.        , 0.47107781])

In [74]:
df = pd.DataFrame(tfidv.transform(docs).toarray(), columns=vocab)
df

,동물원에서,바나나를,봤어,오늘,원숭이에게,원숭이와,줬어,코끼리를
0,0.335176,0.000000,0.471078,0.471078,0.000000,0.471078,0.000000,0.471078
1,0.278943,0.784088,0.000000,0.000000,0.392044,0.000000,0.392044,0.000000


## gensim 사용

In [83]:
import gensim
from gensim import corpora
from gensim.models import TfidfModel

docs = ['오늘 동물원에서 원숭이와 코끼리를 봤어',
       '동물원에서 원숭이에게 바나나를 줬어 바나나를']

doc_ls = [doc.split() for doc in docs]
id2word = corpora.Dictionary(doc_ls)
TDM = [id2word.doc2bow(doc) for doc in doc_ls]
model = TfidfModel(TDM)
tfidf = model[TDM]
tfidf[0]

[(1, 0.5), (2, 0.5), (3, 0.5), (4, 0.5)]

In [84]:
vocab = [id2word[i] for i in id2word.keys()]
TDM_matrix = [sparse2full(doc,len(vocab)).tolist() for doc in tfidf]
pd.DataFrame(TDM_matrix, columns=vocab)

,동물원에서,봤어,오늘,원숭이와,코끼리를,바나나를,원숭이에게,줬어
0,0.0,0.5,0.5,0.5,0.5,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.816497,0.408248,0.408248


In [85]:
tfidf[1]

[(5, 0.8164965809277261), (6, 0.4082482904638631), (7, 0.4082482904638631)]

여기까지 DTM,TDM 그리고 이들의 단점을 보완한 TF-IDF까지 공부해보았다. 이제 기본적인 토대만 안 정도이지만 그래도 이해하고 응용해봤다는 것에 박수를 쳐봅니다. 챡챡챡. 

**-- END **